# Практическая работа №2. Ассоциативный анализ для набора lastfm
1. Выбрать несколько стран для набора данных lastfm для сравнительного анализа

Импортируем необходимые библиотеки.

In [1]:
import pandas as pd
import numpy as np
from  mlxtend import frequent_patterns as ml
from warnings import filterwarnings as fw
fw('ignore')

Импортируем датасет, просматриваем его начало.

In [2]:
data = pd.read_csv('lastfm.csv')
data.head(20)

,user,artist,sex,country
0,1,red hot chili peppers,f,Germany
1,1,the black dahlia murder,f,Germany
2,1,goldfrapp,f,Germany
3,1,dropkick murphys,f,Germany
4,1,le tigre,f,Germany
5,1,schandmaul,f,Germany
6,1,edguy,f,Germany
7,1,jack johnson,f,Germany
8,1,eluveitie,f,Germany
9,1,the killers,f,Germany


Просматриваем распределение данных по странам.

In [3]:
data['country'].value_counts().head(15)

United States         59558
United Kingdom        27638
Germany               24251
Poland                17111
Sweden                12379
Brazil                11922
Russian Federation    11676
Finland               10157
Netherlands            9673
Spain                  9322
Canada                 6928
Australia              6407
France                 5962
Italy                  5717
Turkey                 4762
Name: country, dtype: int64

Выбираем несколько стран (Германия, Испания, Бразилия).

In [4]:
countries = ['Germany','Spain','Brazil']
dataF = data[data['country'].isin(countries)]

2. Подготовить набор для поиска ассоциативных правил (группировка, «бинаризация»)

Просматриваем распределение отфильтрованных данных по полу, странам, исполнителям.

In [5]:
dataF['sex'].value_counts()

m    33501
f    11994
Name: sex, dtype: int64

In [6]:
dataF['country'].value_counts()

Germany    24251
Brazil     11922
Spain       9322
Name: country, dtype: int64

In [7]:
dataF['artist'].value_counts()

coldplay                 409
the beatles              370
red hot chili peppers    314
radiohead                313
linkin park              295
                        ... 
kult                       2
happysad                   1
coma                       1
håkan hellström            1
o.s.t.r.                   1
Name: artist, Length: 1003, dtype: int64

Просмотрим среднее количество групп, которые прослушивает один пользователь.

In [ ]:
dataF.groupby('user')['artist'].apply(';'.join).str.get_dummies(";").apply(sum, axis=1).describe()

Группируем и бинаризируем данные с помощью get_dummies.

In [27]:
dataF.groupby('user')['artist'].apply(';'.join).str.get_dummies(";").apply(sum, axis=1).describe()

count    2393.000000
mean       19.011283
std        10.385961
min         1.000000
25%        10.000000
50%        19.000000
75%        27.000000
max        55.000000
dtype: float64

Группируем и бинаризируем данные с помощью get_dummies.

In [8]:
dataB = dataF.groupby('user')['artist'].apply(';'.join)
dataB = dataB.str.get_dummies(";")
dataB.head()

,...and you will know us by the trail of dead,2pac,3 doors down,30 seconds to mars,311,36 crazyfists,44,50 cent,65daysofstatic,[unknown],...,wilco,within temptation,wolfgang amadeus mozart,wu-tang clan,yann tiersen,yeah yeah yeahs,yellowcard,yo la tengo,zero 7,Édith piaf
user,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
33,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
42,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
62,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


Разделим набор данных на поднаборы по полу/стране. 
Их также группируем/бинаризируем.

In [9]:
dataBsF = dataF[dataF['sex']=='f'].groupby('user')['artist'].apply(';'.join).str.get_dummies(";")
dataBsM = dataF[dataF['sex']=='m'].groupby('user')['artist'].apply(';'.join).str.get_dummies(";")
dataBcG = dataF[dataF['country']=='Germany'].groupby('user')['artist'].apply(';'.join).str.get_dummies(";")
dataBcS = dataF[dataF['country']=='Spain'].groupby('user')['artist'].apply(';'.join).str.get_dummies(";")
dataBcB = dataF[dataF['country']=='Brazil'].groupby('user')['artist'].apply(';'.join).str.get_dummies(";")
dataBsFcG = dataF[(dataF['country']=='Germany') & (dataF['sex']=='f')].groupby('user')['artist'].apply(';'.join).str.get_dummies(";")
dataBsMcB = dataF[(dataF['country']=='Brazil') & (dataF['sex']=='m')].groupby('user')['artist'].apply(';'.join).str.get_dummies(";")

3. Найти характерные комбинации музыкальных коллективов. Сопоставить характерные
комбинации для выборок по странам, по полу.

Рассмотрим характерные комбинации для всей выборки.

In [10]:
dataBapr=ml.apriori(dataB, min_support=0.01, use_colnames=True).sort_values('support', ascending=False)
dataBapr['len'] = dataBapr['itemsets'].apply(lambda x: len(x))
dataBapr[dataBapr['len']>1]

,support,itemsets,len
1175,0.047639,"(coldplay, radiohead)",2
1193,0.046803,"(coldplay, the killers)",2
1157,0.043878,"(coldplay, muse)",2
829,0.042206,"(the killers, arctic monkeys)",2
1178,0.042206,"(coldplay, red hot chili peppers)",2
...,...,...,...
699,0.010029,"(alice in chains, nirvana)",2
1536,0.010029,"(incubus, the beatles)",2
1319,0.010029,"(die Ärzte, the hives)",2
2146,0.010029,"(bloc party, foo fighters, arctic monkeys)",3


Рассмотрим характерные комбинации для мужчин/женщин.

In [11]:
dataBsFapr=ml.apriori(dataBsF, min_support=0.01, use_colnames=True).sort_values('support', ascending=False)
dataBsFapr['len'] = dataBsFapr['itemsets'].apply(lambda x: len(x))
dataBsFapr[dataBsFapr['len']>1]

,support,itemsets,len
1441,0.063123,"(coldplay, radiohead)",2
1444,0.061462,"(coldplay, red hot chili peppers)",2
898,0.059801,"(the killers, arctic monkeys)",2
899,0.059801,"(arctic monkeys, the kooks)",2
1463,0.059801,"(coldplay, the killers)",2
...,...,...,...
2554,0.011628,"(nofx, the offspring)",2
2550,0.011628,"(the smashing pumpkins, nirvana)",2
2540,0.011628,"(rage against the machine, nirvana)",2
2533,0.011628,"(papa roach, nirvana)",2


In [12]:
dataBsMapr=ml.apriori(dataBsM, min_support=0.01, use_colnames=True).sort_values('support', ascending=False)
dataBsMapr['len'] = dataBsMapr['itemsets'].apply(lambda x: len(x))
dataBsMapr[dataBsMapr['len']>1]

,support,itemsets,len
1145,0.042434,"(coldplay, the killers)",2
1130,0.042434,"(coldplay, radiohead)",2
1116,0.040201,"(coldplay, muse)",2
2016,0.039084,"(the beatles, radiohead)",2
1541,0.038526,"(metallica, iron maiden)",2
...,...,...,...
1467,0.010050,"(hammerfall, iron maiden)",2
1466,0.010050,"(hammerfall, in flames)",2
1462,0.010050,"(guns n roses, the beatles)",2
1452,0.010050,"(green day, ramones)",2


Рассмотрим характерные комбинации для жителей Германии/Бразилии/Испании.

In [13]:
dataBcGapr=ml.apriori(dataBcG, min_support=0.01, use_colnames=True).sort_values('support', ascending=False)
dataBcGapr['len'] = dataBcGapr['itemsets'].apply(lambda x: len(x))
dataBcGapr[dataBcGapr['len']>1]

,support,itemsets,len
1425,0.065235,"(die toten hosen, die Ärzte)",2
1533,0.056484,"(rammstein, die Ärzte)",2
1549,0.056484,"(die Ärzte, system of a down)",2
1535,0.054893,"(die Ärzte, red hot chili peppers)",2
2401,0.053302,"(rammstein, system of a down)",2
...,...,...,...
1124,0.010342,"(rammstein, breaking benjamin)",2
2560,0.010342,"(bloc party, billy talent, arctic monkeys)",3
2562,0.010342,"(billy talent, arctic monkeys, mando diao)",3
2563,0.010342,"(billy talent, the hives, arctic monkeys)",3


In [14]:
dataBcBapr=ml.apriori(dataBcB, min_support=0.01, use_colnames=True).sort_values('support', ascending=False)
dataBcBapr['len'] = dataBcBapr['itemsets'].apply(lambda x: len(x))
dataBcBapr[dataBcBapr['len']>1]

,support,itemsets,len
2928,0.079365,"(the beatles, radiohead)",2
2474,0.077778,"(los hermanos, radiohead)",2
2487,0.077778,"(the beatles, los hermanos)",2
1494,0.076190,"(the beatles, chico buarque)",2
2943,0.069841,"(radiohead, the strokes)",2
...,...,...,...
3268,0.011111,"(the beatles, queen, aerosmith)",3
3267,0.011111,"(the beatles, pink floyd, aerosmith)",3
3266,0.011111,"(pink floyd, queen, aerosmith)",3
3262,0.011111,"(metallica, van halen, aerosmith)",3


In [15]:
dataBcSapr=ml.apriori(dataBcS, min_support=0.01, use_colnames=True).sort_values('support', ascending=False)
dataBcSapr['len'] = dataBcSapr['itemsets'].apply(lambda x: len(x))
dataBcSapr[dataBcSapr['len']>1]

,support,itemsets,len
1176,0.088933,"(coldplay, the killers)",2
1135,0.079051,"(coldplay, muse)",2
1155,0.077075,"(coldplay, radiohead)",2
2058,0.075099,"(muse, the killers)",2
1498,0.069170,"(the killers, franz ferdinand)",2
...,...,...,...
4341,0.011858,"(the kooks, editors, arctic monkeys, franz fer...",4
4343,0.011858,"(kasabian, interpol, editors, arctic monkeys)",4
4344,0.011858,"(mando diao, interpol, editors, arctic monkeys)",4
4346,0.011858,"(interpol, oasis, editors, arctic monkeys)",4


4. Сравнить результаты поиска характерных комбинаций разных алгоритмов (apriori,
fpgrowth, fpmax)

Рассмотрим характерные комбинации для Бразилии и сравним с результатом, полученным с помощью Apriori.

In [16]:
dataBcBfpg = ml.fpgrowth(dataBcB, min_support=0.015, use_colnames=True).sort_values('support', ascending=False)
dataBcBfpg['len'] = dataBcBfpg['itemsets'].apply(lambda x: len(x))
dataBcBfpg[dataBcBfpg['len']>1].head(10)

,support,itemsets,len
832,0.079365,"(the beatles, radiohead)",2
830,0.077778,"(los hermanos, radiohead)",2
1075,0.077778,"(the beatles, los hermanos)",2
842,0.076190,"(the beatles, chico buarque)",2
853,0.069841,"(radiohead, the strokes)",2
841,0.068254,"(chico buarque, los hermanos)",2
463,0.066667,"(the beatles, arctic monkeys)",2
1527,0.065079,"(the beatles, amy winehouse)",2
1211,0.065079,"(the beatles, pink floyd)",2
826,0.063492,"(the beatles, coldplay)",2


In [17]:
dataBcBfpm = ml.fpmax(dataBcB, min_support=0.05, use_colnames=True).sort_values('support', ascending=False)
dataBcBfpm['len'] = dataBcBfpm['itemsets'].apply(lambda x: len(x))
dataBcBfpm[dataBcBfpm['len']>1].head(10)

,support,itemsets,len
97,0.079365,"(the beatles, radiohead)",2
102,0.077778,"(the beatles, los hermanos)",2
96,0.077778,"(los hermanos, radiohead)",2
94,0.076190,"(the beatles, chico buarque)",2
86,0.069841,"(radiohead, the strokes)",2
93,0.068254,"(chico buarque, los hermanos)",2
91,0.066667,"(the beatles, arctic monkeys)",2
82,0.065079,"(pink floyd, the beatles)",2
101,0.065079,"(the beatles, amy winehouse)",2
99,0.063492,"(the beatles, coldplay)",2


Результаты получили практически идентичные, отличие может быть только в порядке комбинаций с одинаковой поддержкой.

5. Построить ассоциативные правила. Вывести правила с наилучшими оценками Confidence,
Lift и др.

In [18]:
dataBcBass=ml.association_rules(dataBcBfpg, metric="confidence", min_threshold=0.75)
dataBcBass.sort_values('confidence', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
70,"(the white stripes, franz ferdinand, radiohead)",(the strokes),0.017460,0.139683,0.017460,1.00,7.159091,0.015021,inf
132,"(avril lavigne, rihanna, paramore)",(the pussycat dolls),0.015873,0.052381,0.015873,1.00,19.090909,0.015042,inf
51,"(gwen stefani, rihanna, avril lavigne)",(the pussycat dolls),0.017460,0.052381,0.017460,1.00,19.090909,0.016546,inf
108,"(gwen stefani, beyoncé, rihanna)",(the pussycat dolls),0.015873,0.052381,0.015873,1.00,19.090909,0.015042,inf
10,"(gwen stefani, rihanna)",(the pussycat dolls),0.022222,0.052381,0.022222,1.00,19.090909,0.021058,inf
...,...,...,...,...,...,...,...,...,...
20,"(beyoncé, paramore)",(rihanna),0.025397,0.076190,0.019048,0.75,9.843750,0.017113,3.695238
19,"(rihanna, paramore)",(beyoncé),0.025397,0.063492,0.019048,0.75,11.812500,0.017435,3.746032
18,"(rihanna, paramore)",(the pussycat dolls),0.025397,0.052381,0.019048,0.75,14.318182,0.017717,3.790476
30,"(foo fighters, ramones)",(the beatles),0.025397,0.255556,0.019048,0.75,2.934783,0.012557,2.977778


In [19]:
dataBcBass.sort_values('lift', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
53,"(gwen stefani, rihanna)","(the pussycat dolls, avril lavigne)",0.022222,0.026984,0.017460,0.785714,29.117647,0.016861,4.540741
52,"(the pussycat dolls, rihanna, avril lavigne)",(gwen stefani),0.022222,0.036508,0.017460,0.785714,21.521739,0.016649,4.496296
109,"(gwen stefani, beyoncé)","(the pussycat dolls, rihanna)",0.020635,0.036508,0.015873,0.769231,21.070234,0.015120,4.175132
10,"(gwen stefani, rihanna)",(the pussycat dolls),0.022222,0.052381,0.022222,1.000000,19.090909,0.021058,inf
51,"(gwen stefani, rihanna, avril lavigne)",(the pussycat dolls),0.017460,0.052381,0.017460,1.000000,19.090909,0.016546,inf
...,...,...,...,...,...,...,...,...,...
93,"(van halen, deep purple)",(the beatles),0.019048,0.255556,0.015873,0.833333,3.260870,0.011005,4.466667
33,"(david bowie, foo fighters)",(the beatles),0.023810,0.255556,0.019048,0.800000,3.130435,0.012963,3.722222
23,"(chico buarque, foo fighters)",(the beatles),0.023810,0.255556,0.019048,0.800000,3.130435,0.012963,3.722222
30,"(foo fighters, ramones)",(the beatles),0.025397,0.255556,0.019048,0.750000,2.934783,0.012557,2.977778


In [20]:
dataBcBass.sort_values('support', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(britney spears, rihanna)",(beyoncé),0.031746,0.063492,0.026984,0.850000,13.387500,0.024969,6.243386
1,"(britney spears, beyoncé)",(rihanna),0.034921,0.076190,0.026984,0.772727,10.142045,0.024324,4.064762
2,"(the white stripes, the strokes)",(radiohead),0.033333,0.173016,0.025397,0.761905,4.403670,0.019630,3.473333
3,"(the doors, led zeppelin)",(pink floyd),0.033333,0.131746,0.025397,0.761905,5.783133,0.021005,3.646667
4,"(the doors, pink floyd)",(led zeppelin),0.028571,0.112698,0.025397,0.888889,7.887324,0.022177,7.985714
...,...,...,...,...,...,...,...,...,...
104,"(kiss, deep purple)",(metallica),0.017460,0.128571,0.015873,0.909091,7.070707,0.013628,9.585714
105,"(deep purple, aerosmith)",(metallica),0.015873,0.128571,0.015873,1.000000,7.777778,0.013832,inf
106,"(gwen stefani, beyoncé)",(rihanna),0.020635,0.076190,0.015873,0.769231,10.096154,0.014301,4.003175
107,"(gwen stefani, the pussycat dolls, beyoncé)",(rihanna),0.017460,0.076190,0.015873,0.909091,11.931818,0.014543,10.161905


6. Найти характерные комбинации и ассоциативные правила длины не менее 5 признаков.
Сравнить оценки качества таких правил с короткими правилами.

Найдем характерные комбинации длиной 5 и более признаков для всех мужчин.

In [21]:
dataBsFapr5 = ml.fpgrowth(dataBsF, min_support=0.01, use_colnames=True).sort_values('support', ascending=False)
dataBsFapr5['len'] = dataBsFapr5['itemsets'].apply(lambda x: len(x))
dataBsFapr5[dataBsFapr5['len']>=5].head(10)

,support,itemsets,len
759,0.013289,"(coldplay, the strokes, the kooks, the killers...",5
758,0.013289,"(coldplay, the strokes, the kooks, arctic monk...",5
753,0.013289,"(coldplay, the strokes, radiohead, the killers...",5
1926,0.013289,"(coldplay, the strokes, oasis, arctic monkeys,...",5
740,0.013289,"(coldplay, the strokes, radiohead, muse, franz...",5
1904,0.013289,"(coldplay, the kooks, oasis, arctic monkeys, t...",5
702,0.013289,"(the beatles, the strokes, radiohead, the kill...",5
3453,0.011628,"(the beatles, coldplay, radiohead, the killers...",5
3471,0.011628,"(coldplay, the kooks, arctic monkeys, the kill...",5
3475,0.011628,"(the beatles, coldplay, the kooks, the killers...",5


In [22]:
dataBsFass5 = ml.association_rules(dataBsFapr5, metric="confidence", min_threshold=0.005)
dataBsFass5['len1'] = dataBsFass5['antecedents'].apply(lambda x: len(x))
dataBsFass5['len2'] = dataBsFass5['consequents'].apply(lambda x: len(x))
dataBsFass5 = dataBsFass5[dataBsFass5['len1'] + dataBsFass5['len2']>=5]
dataBsFass5.sort_values('confidence', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,len1,len2
15262,"(the beatles, coldplay, the strokes, the kille...",(radiohead),0.011628,0.127907,0.011628,1.000000,7.818182,0.010141,inf,5,1
12121,"(coldplay, the strokes, the kooks, oasis, the ...",(arctic monkeys),0.011628,0.139535,0.011628,1.000000,7.166667,0.010005,inf,5,1
10083,"(the beatles, the killers, the kooks, franz fe...",(coldplay),0.011628,0.214286,0.011628,1.000000,4.666667,0.009136,inf,4,1
15263,"(the beatles, coldplay, radiohead, the killers...",(the strokes),0.011628,0.104651,0.011628,1.000000,9.555556,0.010411,inf,5,1
15261,"(the beatles, coldplay, the strokes, radiohead...",(the killers),0.011628,0.144518,0.011628,1.000000,6.919540,0.009947,inf,5,1
...,...,...,...,...,...,...,...,...,...,...,...
16399,(coldplay),"(interpol, muse, the strokes, arctic monkeys)",0.214286,0.013289,0.011628,0.054264,4.083333,0.008780,1.043326,1,4
15317,(coldplay),"(the beatles, the strokes, radiohead, the kill...",0.214286,0.013289,0.011628,0.054264,4.083333,0.008780,1.043326,1,5
11535,(coldplay),"(oasis, the strokes, the kooks, the killers)",0.214286,0.011628,0.011628,0.054264,4.666667,0.009136,1.045082,1,4
11690,(coldplay),"(the beatles, arctic monkeys, oasis, the kooks)",0.214286,0.011628,0.011628,0.054264,4.666667,0.009136,1.045082,1,4


7. Привести примеры «тривиальных» и «нетривиальных» правил

Тривиальные правила: 
(beyoncé, britney spears) -> (rihanna), (pink floyd, the doors) -> (led zeppelin)

Нетривиальные правила: 
(foo fighters, ramones) -> (the beatles), (chico buarque, foo fighters) -> (the beatles)	